In [1]:
import os

In [2]:
models_path = os.path.join("Tensorflow","models")

In [3]:
!mkdir {models_path}

A subdirectory or file Tensorflow\models already exists.


In [4]:
import wget

In [5]:
import object_detection

In [6]:
labels = [{'name':'bottle', 'id':1}, {'name':'wallet', 'id':2}, {'name':'vivek', 'id':3}, {'name':'id', 'id':4}]
l = os.path.join('tensorflow','workspace','annotations','label_map.pbtxt')

with open(l, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

python generate_tfrecord.py -x C:\Users\vivek\Desktop\1\tensorflow\workspace\train -i C:\Users\vivek\Desktop\1\tensorflow\workspace\train -l C:\Users\vivek\Desktop\1\tensorflow\workspace\annotations\label_map.pbtxt -o C:\Users\vivek\Desktop\1\tensorflow\workspace\Data\train.record

python generate_tfrecord.py -x C:\Users\vivek\Desktop\1\tensorflow\workspace\test -i C:\Users\vivek\Desktop\1\tensorflow\workspace\test -l C:\Users\vivek\Desktop\1\tensorflow\workspace\annotations\label_map.pbtxt -o C:\Users\vivek\Desktop\1\tensorflow\workspace\Data\test.record

In [7]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [8]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file('tensorflow//workspace//mymodel//pipeline.config')
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join('tensorflow//workspace//mymodel','ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [9]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [10]:
category_index = label_map_util.create_category_index_from_labelmap("Tensorflow//workspace//annotations//label_map.pbtxt")

In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

python model_main_tf2.py --model_dir mymodel --num_train_steps 5000 --pipeline_config_path mymodel\pipeline.config
python model_main_tf2.py --model_dir mymodel --num_train_steps 10000 --pipeline_config_path mymodel\pipeline.config --checkpoint_dir mymodel

tensorboard --logdir .